In [1]:
import numpy as np
import pandas as pd
import os
from rdkit import Chem, ML
from rdkit.ML import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
from rdkit.Chem import SaltRemover
import oddt
from oddt import virtualscreening
from oddt.virtualscreening import electroshape

In [2]:
# Read in the chembl data: contains chembl_id, pchembl_value, assay_type and canonical_smiles – among other columns
file_name = 'cah2_extracted_chembl_data_ours.csv'
df = pd.read_csv(os.path.join('../DSProject/data',file_name)) # ! diff path
## select columns of interest
### note that for all columns, standard_type='Ki', standard_relation='=', data_validity_comment=NaN, potential_duplicate=0, assay_type='B'
df = df[['target_chembl_id','molecule_chembl_id','assay_chembl_id','pchembl_value','canonical_smiles','standard_type','assay_type','description']]
# df

In [3]:
# Identify duplicated entries
# df = df.sort_values(by='molecule_chembl_id')
# df[df.duplicated(subset=['molecule_chembl_id'])]

# Obtain the mean pchembl_value of duplicated entries – all entries are Ki
# df.groupby('molecule_chembl_id')['pchembl_value'].mean()
df = df.groupby(['target_chembl_id','molecule_chembl_id','canonical_smiles','standard_type','assay_type']).mean().reset_index()
df

,target_chembl_id,molecule_chembl_id,canonical_smiles,standard_type,assay_type,pchembl_value
0,CHEMBL205,CHEMBL100075,CC(C)c1cc(-c2ccccc2)cc(C(C)C)[n+]1CC(=O)OCCOc1...,Ki,B,8.15
1,CHEMBL205,CHEMBL100266,CN(C)CCOC(=O)c1cccc(S(=O)(=O)Nc2nnc(S(N)(=O)=O...,Ki,B,8.68
2,CHEMBL205,CHEMBL100329,CCN(CC)CCNC(=O)c1cccc(S(=O)(=O)Nc2nnc(S(N)(=O)...,Ki,B,8.77
3,CHEMBL205,CHEMBL100456,CCc1cc(-c2ccccc2)cc(CC)[n+]1CC(=O)Oc1ccc2nc(S(...,Ki,B,8.15
4,CHEMBL205,CHEMBL100580,Cc1cc(C)[n+](CC(=O)NNc2ccc(S(N)(=O)=O)cc2)c(C)...,Ki,B,6.50
...,...,...,...,...,...,...
4961,CHEMBL205,CHEMBL99697,CCc1cc(-c2ccccc2)cc(CC)[n+]1CC(=O)Nc1ccc(S(N)(...,Ki,B,7.82
4962,CHEMBL205,CHEMBL99736,Cc1cc(C)[n+](CC(=O)NCCC(=O)Nc2nnc(S(N)(=O)=O)s...,Ki,B,8.15
4963,CHEMBL205,CHEMBL99855,Cc1cc(-c2ccccc2)cc(C)[n+]1CC(=O)NCCC(=O)Nc1nnc...,Ki,B,8.40
4964,CHEMBL205,CHEMBL99927,COCCOC(=O)c1ccc(S(=O)(=O)Nc2nnc(S(N)(=O)=O)s2)cc1,Ki,B,8.55


In [4]:
# Compute features

In [5]:
## Compute molecular descriptors using RDKit

### Iterate over rows of the chembl dataframe – add row number and molecule info (obtained via Chem.MolFromSmiles) to mols
mols = []
for i, row in df.iterrows():
    mols.append(Chem.MolFromSmiles(row['canonical_smiles']))
df['RDKit_Molecule'] = mols

In [6]:
### Obtain RDKit molecule descriptors

#### List of descriptors – obtained from https://www.rdkit.org/docs/GettingStartedInPython.html#list-of-available-descriptors
simplelist = ['Gasteiger/Marsili Partial Charges', 'BalabanJ', 'BertzCT', 'Ipc', 'HallKierAlpha', 'Kappa1 - Kappa3', 'Chi0, Chi1', 'Chi0n - Chi4n', 'Chi0v - Chi4v', 'MolLogP', 'MolMR', 'MolWt', 'ExactMolWt', 'HeavyAtomCount', 'HeavyAtomMolWt', 'NHOHCount', 'NOCount', 'NumHAcceptors', 'NumHDonors', 'NumHeteroatoms', 'NumRotatableBonds', 'NumValenceElectrons', 'NumAmideBonds', 'Num{Aromatic,Saturated,Aliphatic}Rings', 'Num{Aromatic,Saturated,Aliphatic}{Hetero,Carbo}cycles', 'RingCount', 'FractionCSP3', 'NumSpiroAtoms', 'NumBridgeheadAtoms', 'TPSA', 'LabuteASA', 'PEOE_VSA1 - PEOE_VSA14', 'SMR_VSA1 - SMR_VSA10', 'SlogP_VSA1 - SlogP_VSA12', 'EState_VSA1 - EState_VSA11', 'VSA_EState1 - VSA_EState10', 'MQNs', 'Topliss fragments', 'Autocorr2D', 'BCUT2D']  # In the list add the names of the descriptors required
calculator = MoleculeDescriptors.MolecularDescriptorCalculator(simplelist)

#### Calculate descriptors for each ligand in the df
descriptors = []
for i in range(df.shape[0]):
    descriptors.append(calculator.CalcDescriptors(df['RDKit_Molecule'][i]))

descriptors_df = pd.DataFrame(descriptors)
descriptors_df.columns = simplelist

In [7]:
df_w_rdkit_desc = pd.concat([df, descriptors_df], axis=1).reindex(df.index)
df_w_rdkit_desc

,target_chembl_id,molecule_chembl_id,canonical_smiles,standard_type,assay_type,pchembl_value,RDKit_Molecule,Gasteiger/Marsili Partial Charges,BalabanJ,BertzCT,...,LabuteASA,PEOE_VSA1 - PEOE_VSA14,SMR_VSA1 - SMR_VSA10,SlogP_VSA1 - SlogP_VSA12,EState_VSA1 - EState_VSA11,VSA_EState1 - VSA_EState10,MQNs,Topliss fragments,Autocorr2D,BCUT2D
0,CHEMBL205,CHEMBL100075,CC(C)c1cc(-c2ccccc2)cc(C(C)C)[n+]1CC(=O)OCCOc1...,Ki,B,8.15,<rdkit.Chem.rdchem.Mol object at 0x14529d0a0>,777,9.343417e-07,1620.257731,...,255.299012,777,777,777,777,777,777,777,777,777
1,CHEMBL205,CHEMBL100266,CN(C)CCOC(=O)c1cccc(S(=O)(=O)Nc2nnc(S(N)(=O)=O...,Ki,B,8.68,<rdkit.Chem.rdchem.Mol object at 0x10ded8dc0>,777,2.165407e+00,1032.214622,...,159.807526,777,777,777,777,777,777,777,777,777
2,CHEMBL205,CHEMBL100329,CCN(CC)CCNC(=O)c1cccc(S(=O)(=O)Nc2nnc(S(N)(=O)...,Ki,B,8.77,<rdkit.Chem.rdchem.Mol object at 0x14529d340>,777,2.103708e+00,1063.616183,...,172.973728,777,777,777,777,777,777,777,777,777
3,CHEMBL205,CHEMBL100456,CCc1cc(-c2ccccc2)cc(CC)[n+]1CC(=O)Oc1ccc2nc(S(...,Ki,B,8.15,<rdkit.Chem.rdchem.Mol object at 0x14529d3a0>,777,7.321210e-07,1506.554420,...,224.725698,777,777,777,777,777,777,777,777,777
4,CHEMBL205,CHEMBL100580,Cc1cc(C)[n+](CC(=O)NNc2ccc(S(N)(=O)=O)cc2)c(C)...,Ki,B,6.50,<rdkit.Chem.rdchem.Mol object at 0x14529d400>,777,7.491665e-07,931.218948,...,169.718301,777,777,777,777,777,777,777,777,777
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4961,CHEMBL205,CHEMBL99697,CCc1cc(-c2ccccc2)cc(CC)[n+]1CC(=O)Nc1ccc(S(N)(...,Ki,B,7.82,<rdkit.Chem.rdchem.Mol object at 0x1453424c0>,777,8.299137e-07,1284.022882,...,215.923605,777,777,777,777,777,777,777,777,777
4962,CHEMBL205,CHEMBL99736,Cc1cc(C)[n+](CC(=O)NCCC(=O)Nc2nnc(S(N)(=O)=O)s...,Ki,B,8.15,<rdkit.Chem.rdchem.Mol object at 0x145342520>,777,9.197034e-07,1041.112855,...,189.052626,777,777,777,777,777,777,777,777,777
4963,CHEMBL205,CHEMBL99855,Cc1cc(-c2ccccc2)cc(C)[n+]1CC(=O)NCCC(=O)Nc1nnc...,Ki,B,8.40,<rdkit.Chem.rdchem.Mol object at 0x145342580>,777,8.771663e-07,1317.364746,...,217.744778,777,777,777,777,777,777,777,777,777
4964,CHEMBL205,CHEMBL99927,COCCOC(=O)c1ccc(S(=O)(=O)Nc2nnc(S(N)(=O)=O)s2)cc1,Ki,B,8.55,<rdkit.Chem.rdchem.Mol object at 0x1453425e0>,777,2.115183e+00,984.086089,...,152.796296,777,777,777,777,777,777,777,777,777
